In [1]:
import sys
from pathlib import Path

import cv2
import hdbscan
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap
import numpy as np
import pandas as pd
import seaborn as sns
import umap
from mpl_toolkits.mplot3d import Axes3D
from skimage import measure
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import parc

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# Import path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
from config import *

In [5]:
# Parameters
min_cluster_size = 100
min_intensity = 0.1
random_seed = 1

# Read data

In [6]:
pixel_features = data_meta / 'pixel_intensity.csv'  # location of file
df = pd.read_csv(pixel_features)
pixels = df.iloc[:, :11]
display(pixels.max())

# Scale data
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(pixels)
pixels_scaled = pd.DataFrame(x_scaled, columns=pixels.columns)
display(pixels_scaled.describe())

DAPI                     24117
Phalloidin               61307
WGA                      57667
Concanavadin A           59602
APC                      61245
Cyclin D1                34600
Cyclin E                 54336
EMMPRIN                  60834
WNT-1                    64637
Non-phospho-B-catenin    28759
DKK1                      6677
dtype: int64

,DAPI,Phalloidin,WGA,Concanavadin A,APC,Cyclin D1,Cyclin E,EMMPRIN,WNT-1,Non-phospho-B-catenin,DKK1
count,4.189708e+06,4.189708e+06,4.189708e+06,4.189708e+06,4.189708e+06,4.189708e+06,4.189708e+06,4.189708e+06,4.189708e+06,4.189708e+06,4.189708e+06
mean,5.490950e-02,4.898193e-02,3.293212e-02,4.402110e-02,7.236760e-03,3.255747e-03,1.000895e-02,5.946675e-03,2.468510e-03,2.045527e-02,1.816867e-02
std,1.399681e-01,4.085330e-02,6.341810e-02,9.892625e-02,1.461627e-02,2.681191e-03,2.095746e-02,1.533676e-02,6.028638e-03,2.209952e-02,1.103692e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.386085e-02,0.000000e+00,0.000000e+00,1.730754e-03,1.820809e-03,6.257362e-04,1.479436e-03,3.248913e-04,5.737334e-03,1.078329e-02
50%,0.000000e+00,3.779888e-02,4.075121e-03,0.000000e+00,4.702425e-03,3.005780e-03,5.061101e-03,4.043791e-03,1.345978e-03,1.404778e-02,1.722330e-02
75%,0.000000e+00,6.006352e-02,3.678013e-02,4.766619e-02,8.686423e-03,4.335260e-03,1.153931e-02,7.446494e-03,2.722899e-03,2.555722e-02,2.426239e-02
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [7]:
pixel_dark = pixels_scaled.le(min_intensity).all(axis=1)
display(pixel_dark.value_counts())
pixels_bright = pixels_scaled[~pixel_dark]
display(pixels_bright.head())

True     2924531
False    1265177
dtype: int64

,DAPI,Phalloidin,WGA,Concanavadin A,APC,Cyclin D1,Cyclin E,EMMPRIN,WNT-1,Non-phospho-B-catenin,DKK1
62,0.0,0.024519,0.068237,0.126925,0.002286,0.003324,0.007748,0.000000,0.001686,0.00000,0.004643
63,0.0,0.022594,0.088404,0.143972,0.000000,0.002283,0.000000,0.000049,0.003605,0.00000,0.005691
64,0.0,0.020422,0.067179,0.153032,0.001404,0.006098,0.000000,0.000000,0.003496,0.00000,0.007938
65,0.0,0.016258,0.062393,0.160179,0.000000,0.000694,0.001804,0.002975,0.001733,0.00000,0.015127
66,0.0,0.020356,0.059185,0.152042,0.002384,0.001040,0.000000,0.003058,0.002243,0.00153,0.007788


# Clustering

In [8]:
def clustering_parc(X, small_pop=200):
    parc1 = parc.PARC(X, jac_weighted_edges = False, small_pop = small_pop)
    parc1.run_PARC() 
    labels = parc1.labels
    
    graph = parc1.knngraph_full()
    embedding= parc1.run_umap_hnsw(X, graph, random_state = 1)
    return labels, embedding

In [ ]:
# Clustering
labels, embedding = clustering_parc(pixels_bright.values)

input data has shape 1265177 (samples) x 11 (features)
knn struct was not available, so making one


# Viz

## Cluster visualization of markers expression

In [ ]:
# Get dataframe per cluster
df_per_label = pixels_bright.groupby('label').mean()
df_per_label

In [ ]:
log_norm = LogNorm(vmin=0, vmax=1)

# Plot heat map
fig, ax = plt.subplots(figsize=(10, 10))
ax = sns.heatmap(df_per_label, cbar_kws={
                 'fraction': 0.01}, linewidth=1, cmap="coolwarm")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45,
                   horizontalalignment='right')

In [ ]:
# Get random colormap
vals = np.linspace(0, 1, len(df_per_label))
np.random.seed(random_seed)
np.random.shuffle(vals)
my_cmap = plt.cm.Paired(vals)

# Change to row colors for clustermap
labels = df_per_label.index.to_list()
my_cmap_dict = dict(zip(labels, my_cmap))
row_colors = pd.DataFrame(labels)[0].map(my_cmap_dict)
row_colors.index += 1

# Plot clustermap
ax = sns.clustermap(df_per_label,
                    cbar_kws={'fraction': 0.01},
                    cmap='coolwarm',
                    linewidth=1,
                    row_colors=row_colors,
                    col_cluster=False, 
                    norm = log_norm)
ax.ax_heatmap.set_xticklabels(
    ax.ax_heatmap.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.ax_heatmap.set_xlabel('Marker and location')
ax.ax_heatmap.set_ylabel('Cluster')

In [ ]:
# Plot clustermap
ax = sns.clustermap(df_per_label,
                    cbar_kws={'fraction': 0.01},
                    cmap='coolwarm',
                    linewidth=1,
                    row_colors=row_colors,
                    row_cluster=False, 
                    norm = log_norm)
ax.ax_heatmap.set_xticklabels(
    ax.ax_heatmap.get_xticklabels(), rotation=45, horizontalalignment='right')
ax.ax_heatmap.set_xlabel('Marker and location')
ax.ax_heatmap.set_ylabel('Cluster')

## Pixel location visualization

First we add back the location and condition information in the pixel_bright dataframe

In [ ]:
pixels_bright = pixels_bright.join(df.iloc[:, 11:])
pixels_bright.head()

In [ ]:
condition = 'Fw1'

df_subset = pixels_bright[pixels_bright.Condition == condition]

In [ ]:
# Read mask image
def get_masks(mask_folder):
    '''
    Function to get all mask from mask forlder
    '''
    # Read masks
    masks = {}

    for (dirpath, dirnames, filenames) in os.walk(mask_folder):
        for name in sorted(filenames):
            if "tiff" in name:
                condition = name.split("_")[0]
                masks[condition] = masks.get(condition, {})
                filename = os.path.join(dirpath, name)
            if "cyto" in name:
                img = skimage.io.imread(filename)
                masks[condition]["cyto"] = img
            elif "nuclei" in name:
                img = skimage.io.imread(filename)
                masks[condition]["nuclei"] = img
    return masks


def qc_nuclei(mask_cyto, mask_nuclei):
    '''
    Function to check if cell masks contain nuclei
    '''
    cell = np.zeros((mask_cyto.shape), dtype=np.uint8)
    nuclei = np.zeros((mask_cyto.shape), dtype=np.uint8)
    cyto = np.zeros((mask_cyto.shape), dtype=np.uint8)

    for label in range(1, mask_cyto.max()):
        # Check if cell has nuclei
        cell_mask = np.where(mask_cyto == label, 1, 0).astype(np.uint8)
        maski = cv2.bitwise_and(mask_nuclei, mask_nuclei, mask=cell_mask)

        # If no nuclei detected then pass
        if maski.max() == 0:
            continue

        # Link label accross cell, nuclei, cyto
        cell = np.where(mask_cyto == label, label, cell)
        nuclei = np.where(maski > 0, label, nuclei)
        maski = cv2.subtract(cell_mask, maski)
        cyto = np.where(maski > 0, label, cyto)
    return cell, nuclei, cyto


masks = get_masks(data_mask)
mask, mask_nuclei, _ = qc_nuclei(masks[condition]['cyto'], masks[condition]['nuclei'])
mask_binary = np.where(mask > 0, 1, 0)

In [ ]:
# Create image from pixel location
x_max, y_max = mask.shape

x = df_subset.X.tolist()
y = df_subset.Y.tolist()
values = df_subset.label.tolist()

img = np.zeros((x_max, y_max))
img[x, y] = values

In [ ]:
# Get contour of masks
contours = {}
contours_nuclei = {}
labels = [n for n in np.unique(mask) if n > 0]
for i in labels:
    temp = np.where(mask == i, mask, 0)
    contours[i] = measure.find_contours(temp, 0.1)[0]
    temp = np.where(mask_nuclei == i, mask, 0)
    contours_nuclei[i] = measure.find_contours(temp, 0.1)[0]

In [ ]:
# Get random colormap
bg_color = np.array([[0, 0, 0, 0]])
colors = np.concatenate((bg_color, my_cmap))
my_cmap_bg = plt.cm.colors.ListedColormap(colors)
my_cmap_binary = plt.cm.colors.ListedColormap(['k', 'dimgray'])

# Show contour
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(mask_binary, cmap=my_cmap_binary)
ax.imshow(img, cmap=my_cmap_bg)
for label, contour in contours.items():
    ax.plot(contour[:, 1], contour[:, 0], linewidth=2, c='w')
    ax.plot(contours_nuclei[label][:, 1],
            contours_nuclei[label][:, 0], linewidth=2, c='w')
ax.axis('off')